# SETTINGS

In [52]:
########## LIBRARIES

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats
from scipy.signal import find_peaks

import os
import time
import datetime
import random
import multiprocessing
import pickle
import warnings
import gc
from tqdm import tqdm

In [53]:
########## HELPER FUNCTIONS

!pip install --upgrade dptools
from dptools import *

Requirement already up-to-date: dptools in /Users/kozodoi/miniconda3/envs/py3/lib/python3.7/site-packages (0.3.10)


In [54]:
########## SETTINGS

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('dark_background')
%matplotlib inline
gc.enable()

# DATA IMPORT

In [55]:
# read data
orders = pd.read_csv('../data/prepared/orders_v1.csv', compression = 'gzip')
items  = pd.read_csv('../data/prepared/items_v1.csv',  compression = 'gzip')
print(orders.shape)
print(items.shape)

(2181955, 5)
(10463, 12)


In [56]:
# convert dates
orders['time']       = pd.to_datetime(orders['time'].astype('str'),       infer_datetime_format = True)
items['promotion_0'] = pd.to_datetime(items['promotion_0'].astype('str'), infer_datetime_format = True)
items['promotion_1'] = pd.to_datetime(items['promotion_1'].astype('str'), infer_datetime_format = True)
items['promotion_2'] = pd.to_datetime(items['promotion_2'].astype('str'), infer_datetime_format = True)

# ADD FEATURES: ITEMS

In [57]:
items.head()

,itemID,simulationPrice,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice,promotion_0,promotion_1,promotion_2
0,1,3.43,NaN,1,4.38,1,1,1,8.84,NaT,NaT,NaT
1,2,9.15,NaN,2,3.00,1,2,1,16.92,NaT,NaT,NaT
2,3,14.04,NaN,3,5.00,1,3,1,15.89,NaT,NaT,NaT
3,4,14.10,NaN,2,4.44,1,2,1,40.17,NaT,NaT,NaT
4,5,7.48,NaN,2,2.33,1,1,1,17.04,NaT,NaT,NaT


In [58]:
# price ratio
items['price_ratio'] = items['simulationPrice'] / items['recommendedRetailPrice']
items['price_ratio'].describe()

count    10463.000000
mean         0.957107
std          0.616878
min          0.020094
25%          0.449184
50%          0.813378
75%          1.359211
max          5.678983
Name: price_ratio, dtype: float64

In [71]:
# detailed item category
items['category'] = items['category1'].astype(str) + items['category2'].astype(str) + items['category3'].astype(str)
items['category'] = items['category'].astype(int)
items['category'].nunique()

75

In [59]:
# customer rating ratio per manufacturer
rating_manufacturer = items.groupby('manufacturer')['customerRating'].agg('mean').reset_index()
rating_manufacturer.columns = ['manufacturer', 'mean_customerRating_manufacturer']
items = items.merge(rating_manufacturer, how = 'left', on = 'manufacturer')
items['customerRating_manufacturer_ratio'] = items['customerRating'] / items['mean_customerRating_manufacturer']
del items['mean_customerRating_manufacturer']
items['customerRating_manufacturer_ratio'].describe()

count    3346.000000
mean        1.000000
std         0.231363
min         0.211680
25%         0.944197
50%         1.060606
75%         1.136024
max         1.767617
Name: customerRating_manufacturer_ratio, dtype: float64

In [73]:
# customer rating ratio per category
rating_category = items.groupby('category')['customerRating'].agg('mean').reset_index()
rating_category.columns = ['category', 'mean_customerRating_category']
items = items.merge(rating_category, how = 'left', on = 'category')
items['customerRating_category_ratio'] = items['customerRating'] / items['mean_customerRating_category']
del items['mean_customerRating_category']
items['customerRating_category_ratio'].describe()

count    3346.000000
mean        1.000000
std         0.233089
min         0.211423
25%         0.927767
50%         1.098585
75%         1.125704
max         1.714286
Name: customerRating_category_ratio, dtype: float64

# ADD FEATURES: ORDERS

In [74]:
# total orders
print(orders['order'].sum())

2715559


In [75]:
##### AGGREGATE ORDERS BY DAY

# aggregation
orders['day_of_year'] = orders['time'].dt.dayofyear
orders_price = orders.groupby(['itemID', 'day_of_year'])['salesPrice'].agg('mean').reset_index()
orders = orders.groupby(['itemID', 'day_of_year'])['order'].agg('sum').reset_index()
orders.head()

,itemID,day_of_year,order
0,1,23,1
1,1,25,1
2,1,29,307
3,1,30,3
4,1,31,1


In [76]:
# check total orders
print(orders['order'].sum())

2715559


In [77]:
##### ADD MISSING INPUTS [ORDERS]

# add items that were never sold before
missing_itemIDs = set(items['itemID'].unique()) - set(orders['itemID'].unique())
missing_rows = pd.DataFrame({'itemID':     list(missing_itemIDs), 
                            'day_of_year': np.ones(len(missing_itemIDs)).astype('int'), 
                            'order':       np.zeros(len(missing_itemIDs)).astype('int')})
orders = pd.concat([orders, missing_rows], axis = 0)
print(orders.shape)

# add zeros for days with no transactions
agg_orders = orders.groupby(['itemID', 'day_of_year']).order.unique().unstack('day_of_year').stack('day_of_year', dropna = False)
agg_orders = agg_orders.reset_index()
agg_orders.columns = ['itemID', 'day_of_year', 'order']
agg_orders['order'].fillna(0, inplace = True)
agg_orders['order'] = agg_orders['order'].astype(int)
print(agg_orders.shape)

(100771, 3)
(1883340, 3)


In [78]:
##### ADD MISSING INPUTS [PRICES]

# add items that were never sold before
missing_rows = pd.DataFrame({'itemID':     list(missing_itemIDs), 
                            'day_of_year': np.ones(len(missing_itemIDs)).astype('int'), 
                            'salesPrice':  np.zeros(len(missing_itemIDs)).astype('int')})
orders_price = pd.concat([orders_price, missing_rows], axis = 0)
print(orders_price.shape)

# add zeros for days with no transactions
agg_orders_price = orders_price.groupby(['itemID', 'day_of_year']).salesPrice.unique().unstack('day_of_year').stack('day_of_year', dropna = False)
agg_orders_price = agg_orders_price.reset_index()
agg_orders_price.columns = ['itemID', 'day_of_year', 'salesPrice']
agg_orders_price['salesPrice'].fillna(0, inplace = True)
agg_orders_price['salesPrice'] = agg_orders_price['salesPrice'].astype(int)
agg_orders_price['salesPrice'][agg_orders_price['salesPrice'] == 0] = np.nan
print(agg_orders_price.shape)

# fill missing prices for dates with no orders
agg_orders_price['salesPrice'] = agg_orders_price.groupby(['itemID']).salesPrice.fillna(method = 'ffill')
agg_orders_price['salesPrice'] = agg_orders_price.groupby(['itemID']).salesPrice.fillna(method = 'bfill')
agg_orders_price = agg_orders_price.merge(items[['itemID', 'simulationPrice']], how = 'left', on = 'itemID')
agg_orders_price['salesPrice'][agg_orders_price['salesPrice'].isnull()] = agg_orders_price['simulationPrice'][agg_orders_price['salesPrice'].isnull()]
del agg_orders_price['simulationPrice']

# merge prices to orders
agg_orders = agg_orders.merge(agg_orders_price, how = 'left', on = ['itemID', 'day_of_year'])
print(agg_orders.shape)

(100771, 3)
(1883340, 3)
(1883340, 4)


In [ ]:
##### FIND PROMOTION DAYS

# documentation says that we have to manually mark promotion days in the training period
# without marking, predictions are difficult because orders in some daye explode without apparent reason
# we need to be very careful with threshold and rely on visuzal analysis and/or better metric to mark promos
# I started with treating promotions as days when the number of orders for a given item exceeds 90th percentile
# now I am trying to use a smarte technique using find_peaks() and specifying height and prominence

# computations
agg_orders['promotion'] = 0
for itemID in tqdm(agg_orders['itemID'].unique()):
    '''
    # using quantiles
    promo_quant = orders[orders['itemID'] == itemID]['order'].quantile(0.90)
    orders.loc[(orders['itemID'] == itemID) & (orders['order'] >= promo_quant), 'promotion'] = 1
    '''
    # using find_peaks
    promo    = np.zeros(len(agg_orders[agg_orders['itemID'] == itemID]))
    avg      = agg_orders[(agg_orders['itemID'] == itemID)]['order'].median()
    std      = agg_orders[(agg_orders['itemID'] == itemID)]['order'].std()
    peaks, _ = find_peaks(np.append(agg_orders[agg_orders['itemID'] == itemID]['order'].values, avg), # append avg to enable marking last point as promo
                          prominence = max(5, std),  # peak difference with neighbor points; max(5,std) to exclude cases when std is too small
                          height     = avg + 2*std)  # minimal height of a peak
    promo[peaks] = 1
    agg_orders.loc[agg_orders['itemID'] == itemID, 'promotion'] = promo
    
# check total promotions
print(agg_orders['promotion'].sum())

 37%|███▋      | 3831/10463 [01:48<02:58, 37.13it/s]

In [ ]:
##### COMPARE PROMOTIONS NUMBER

# computations
promo_in_train = (agg_orders['promotion'].sum() / agg_orders['day_of_year'].max()) / len(items)
promo_in_test  = (3*len(items) - items.promotion_0.isnull().sum() - items.promotion_2.isnull().sum() - items.promotion_1.isnull().sum()) / 14 / len(items)

# info
print('Daily p(promotion) per item in train: {}'.format(np.round(promo_in_train, 4)))
print('Daily p(promotion) per item in test:  {}'.format(np.round(promo_in_test , 4)))

In [ ]:
### EXAMPLE SALES PLOT

# compute promo count
promo_count = agg_orders.groupby('itemID')['promotion'].agg('sum').reset_index()
promo_count = promo_count.sort_values('promotion').reset_index(drop = True)

# plot some items
item_plots = [0, 2000, 4000, 6000, 8000, 9000, 10000, 10100, 10200, 10300, 10400, 10462]
fig = plt.figure(figsize = (16, 12))
for i in range(len(item_plots)):
    plt.subplot(3, 4, i + 1)
    df = agg_orders[agg_orders.itemID == promo_count['itemID'][item_plots[i]]]
    plt.scatter(df['day_of_year'], df['order'], c = df['promotion'])
    plt.ylabel('Total Orders')
    plt.xlabel('Day')

In [15]:
##### COMPUTING TARGETS AND FEATURES

# parameters
days_input  = [1, 3, 7, 14, 21, 28]
days_target = 14

# preparations
day_first = np.max(days_input)
day_last  = agg_orders['day_of_year'].max() - days_target
orders    = None

# merge manufacturer and brand
agg_orders = agg_orders.merge(items[['itemID', 'manufacturer']], how = 'left')

# computations
for day_of_year in tqdm(range(day_first, day_last)):
                
    ###### VALIDAION: TARGET, PROMOTIONS, PRICES
        
    # day intervals
    target_day_min = day_of_year + 1
    target_day_max = day_of_year + days_target
    
    # compute variables: labeled data
    if day_of_year < agg_orders['day_of_year'].max():
        
        # target and promo
        tmp_df = agg_orders[(agg_orders['day_of_year'] >= target_day_min) &
                            (agg_orders['day_of_year'] <= target_day_max)
                           ].groupby('itemID')['order', 'promotion'].agg('sum').reset_index()
        tmp_df.columns = ['itemID', 'target', 'promo_in_test']
        
        # price
        tmp_df['mean_price'] = agg_orders[(agg_orders['day_of_year'] >= target_day_min) &
                                          (agg_orders['day_of_year'] <= target_day_max)
                                         ].groupby('itemID')['salesPrice'].agg('mean').reset_index()['salesPrice']
        
    # compute variables: unlabeled data
    else:
        
        # target and promo
        tmp_df = agg_orders[agg_orders['day_of_year'] == day_of_year].groupby('itemID')['order', 'promotion'].agg('sum').reset_index()
        tmp_df.columns = ['itemID', 'target', 'promo_in_test']        
        tmp_df['target'] = np.nan
        
        # price
        tmp_df = tmp_df.merge(items[['itemID', 'simulationPrice']], how = 'left', on = 'itemID')
        tmp_df.columns = ['itemID', 'target', 'promo_in_test', 'mean_price']      
    
        
    ###### TRAINING: LAG-BASED FEATURES
        
    # compute features
    for day_input in days_input:
        
        # day intervals
        input_day_min  = day_of_year - day_input + 1
        input_day_max  = day_of_year
    
        # frequency, promo and price
        tmp_df_input = agg_orders[(agg_orders['day_of_year'] >= input_day_min) &
                                  (agg_orders['day_of_year'] <= input_day_max)
                                 ].groupby('itemID')
        tmp_df['order_sum_last_'   + str(day_input)] = tmp_df_input['order'].agg('sum').reset_index()['order']
        tmp_df['order_count_last_' + str(day_input)] = tmp_df_input['order'].agg(lambda x: len(x[x > 0])).reset_index()['order']
        tmp_df['promo_count_last_' + str(day_input)] = tmp_df_input['promotion'].agg('sum').reset_index()['promotion']
        tmp_df['mean_price_last_'  + str(day_input)] = tmp_df_input['salesPrice'].agg('mean').reset_index()['salesPrice']

        # frequency, promo per manufacturer
        tmp_df_input = agg_orders[(agg_orders['day_of_year'] >= input_day_min) &
                                  (agg_orders['day_of_year'] <= input_day_max)
                                 ].groupby('manufacturer')
        tmp_df['order_manufacturer_sum_last_'   + str(day_input)] = tmp_df_input['order'].agg('sum').reset_index()['order']
        tmp_df['order_manufacturer_count_last_' + str(day_input)] = tmp_df_input['order'].agg(lambda x: len(x[x > 0])).reset_index()['order']
        tmp_df['promo_manufacturer_count_last_' + str(day_input)] = tmp_df_input['promotion'].agg('sum').reset_index()['promotion']

        # frequency, promo per all items
        tmp_df_input = agg_orders[(agg_orders['day_of_year'] >= input_day_min) &
                                  (agg_orders['day_of_year'] <= input_day_max)]
        tmp_df['order_all_sum_last_'   + str(day_input)] = tmp_df_input['order'].agg('sum')
        tmp_df['order_all_count_last_' + str(day_input)] = tmp_df_input['order'].agg(lambda x: len(x[x > 0]))
        tmp_df['promo_all_count_last_' + str(day_input)] = tmp_df_input['promotion'].agg('sum')
        
        # recency
        if day_input == max(days_input):
            tmp_df_input = agg_orders[(agg_orders['day_of_year'] >= input_day_min) &
                                      (agg_orders['day_of_year'] <= input_day_max) &
                                      (agg_orders['order'] > 0)
                                     ].groupby('itemID')
            tmp_df['days_since_last_order'] = (day_of_year - tmp_df_input['day_of_year'].agg('max')).reindex(tmp_df.itemID).reset_index()['day_of_year']
            tmp_df['days_since_last_order'].fillna(day_input, inplace = True)
            
            
    # add day of year
    tmp_df.insert(1, column = 'day_of_year', value = day_of_year)
        
    # merge data
    orders = pd.concat([orders, tmp_df], axis = 0)

  0%|          | 0/138 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
##### RECOMPUTE MEAN PRICE RATIOS

print(orders.shape)
for var in ['mean_price_last_3'. 'mean_price_last_3', 'mean_price_last_7', 'mean_price_last_14', 'mean_price_last_21', 'mean_price_last_28']:
    orders['ratio' + str(var)] = orders['mean_price'] / orders[var]
    del orders[var]
del orders['mean_price']
print(orders.shape)

In [ ]:
# example sales plot
df = orders[orders.itemID == 1]
plt.figure(figsize = (10, 5))
plt.scatter(df['day_of_year'], df['target'], c = df['promo_in_test'])
plt.title('itemID == 1')
plt.ylabel('Target (orders in next 14 days)')
plt.xlabel('Day')

# MERGE DATA SETS

In [ ]:
print(orders.shape)
print(items.shape)
df = pd.merge(orders, items, on = 'itemID', how = 'left')
print(df.shape)

# EXTRACT TEST DATA

In [ ]:
# partition intro train and test
df_train = df[df['day_of_year'] <  df['day_of_year'].max()]
df_test  = df[df['day_of_year'] == df['day_of_year'].max()]
print(df_train.shape)
print(df_test.shape)

In [ ]:
# add promotion info to test
promo_vars = df_test.filter(like = 'promotion_').columns
df_test['promo_in_test'] = 3 - df_test[promo_vars].isnull().sum(axis = 1)
df_test['promo_in_test'].describe()

# drop promotion dates
df_test.drop(promo_vars,  axis = 1, inplace = True)
df_train.drop(promo_vars, axis = 1, inplace = True)
print(df_train.shape)
print(df_test.shape)

# EXPORT

In [ ]:
##### LOGS

# data (shape): descriprion
#
# df_v1 (1357920, 23): first aggregated df version with 4x2 lagged frequency-based features [7,14,21,28 days]
# df_v2 (1357920, 24): added promo_in_test feature using 0.9 percentile of item-specific demand
# df_v3 (1433431, 21): added rows with missing itemIDs for items that were never sold during training period;
#                      added test sample with NA target and promo_in_test feature
#                      removed promotion_i features info from training data
# df_v4 (1433431, 26): added promo_count features; added days_since_last_order with NA set to time horizon
# df_v5 (1433431, 34): added all_promo_count and all_order_count features for aggregated stats on all items
# df_v6 (1433431, 34): changed technique to identify promotions using find_peaks()
# df_v7 (1433431, 38): changed names of "order_all_..." features, added sum of all orders
# df_v8 (1433431, 50): added frequency and promo features per manufacturer
# df_v9 (1433431, 54): added mean_price ratio features
# df_v10(1433431, 64): added all lag features for yesterday (day = 1)

In [ ]:
# save data frame
# save_csv_version() automatically adds version number to prevent overwriting
save_csv_version('../data/prepared/df.csv',      df_train, index = False, compression = 'gzip')
save_csv_version('../data/prepared/df_test.csv', df_test,  index = False, compression = 'gzip', min_version = 3)
print(df_train.shape)
print(df_test.shape)